Loading the python packages

In [49]:
import pandas as pd
from pickle import load
import plotly.graph_objects as go

Loading the Datasets from Trusted Layer

In [50]:
crypto_ai = pd.read_csv('Trusted/crypto_ai_energy.csv')
world = pd.read_csv('Trusted/world_energy.csv')

Defining the forecast range

In [51]:
years_ahead = 12
start_year = 2024
years_predict = pd.DataFrame({'Year':range(start_year,start_year+years_ahead,1)})

Loading the Models from Models folder

In [52]:
# load the model from disk
crypto_ai_model = load(open('Models/crypto_ai_forecast.sav', 'rb'))
world_model = load(open('Models/world_energy_forecast.sav', 'rb'))
#rescaledValidationX = scaler.transform(X_validation) #in case the data is scaled.
#predictions = model.predict(rescaledValidationX)
predictions_cd = crypto_ai_model.get_forecast(steps = years_ahead, exog = years_predict)
predictions_w = world_model.get_forecast(steps = years_ahead, exog = years_predict)

Crypto and AI Energy Consumption Forecast

In [55]:
forecast_cd = pd.concat([years_predict, predictions_cd.predicted_mean.reset_index(drop='True'), predictions_cd.conf_int().reset_index(drop='True')], axis=1)
forecast_cd.head()

,Year,predicted_mean,lower Total Consumption,upper Total Consumption
0,2024,578.789343,539.707296,617.871389
1,2025,621.633935,578.342660,664.925210
2,2026,646.814561,601.871433,691.757689
3,2027,663.836336,619.238299,708.434374
4,2028,675.320070,629.242795,721.397346


In [57]:
# Create Plotly figure
fig = go.Figure()

# Observed data
fig.add_trace(go.Scatter(
    x=crypto_ai['Year'], y=crypto_ai['Total Consumption'],
    mode='lines', name='Observed',
    line=dict(color='black')
))

# Mean forecast
fig.add_trace(go.Scatter(
    x=forecast_cd['Year'], y=forecast_cd['predicted_mean'],
    mode='lines', name='Forecast',
    line=dict(color='blue')
))

# Confidence interval (as shaded area)
fig.add_trace(go.Scatter(
    x=forecast_cd['Year'].tolist() + forecast_cd['Year'].tolist()[::-1], # x for upper and then lower reversed
    y=forecast_cd['lower Total Consumption'].tolist() + forecast_cd['upper Total Consumption'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% CI'
))

fig.update_layout(
    title="Crypto and AI Energy Consumption - ARIMA Forecast with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()

World Energy Consumption Forecast

In [58]:
forecast_w = pd.concat([years_predict, predictions_w.predicted_mean.reset_index(drop='True'), predictions_w.conf_int().reset_index(drop='True')], axis=1)
forecast_w.head()

,Year,predicted_mean,lower Consumption,upper Consumption
0,2024,117943.343603,114828.922697,121057.764509
1,2025,120252.371526,115785.609388,124719.133665
2,2026,121537.483057,116542.941868,126532.024246
3,2027,121621.523628,116183.097026,127059.950230
4,2028,124272.643336,118464.769252,130080.517420


In [60]:
# Create Plotly figure
fig = go.Figure()

# Observed data
fig.add_trace(go.Scatter(
    x=world['Year'], y=world['Consumption'],
    mode='lines', name='Observed',
    line=dict(color='black')
))

# Mean forecast
fig.add_trace(go.Scatter(
    x=forecast_w['Year'], y=forecast_w['predicted_mean'],
    mode='lines', name='Forecast',
    line=dict(color='blue')
))

# Confidence interval (as shaded area)
fig.add_trace(go.Scatter(
    x=forecast_w['Year'].tolist() + forecast_w['Year'].tolist()[::-1], # x for upper and then lower reversed
    y=forecast_w['lower Consumption'].tolist() + forecast_w['upper Consumption'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% CI'
))

fig.update_layout(
    title="Total World Energy Consumption - ARIMA Forecast with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()